### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Read a CSV file

In [3]:
sg_mrt = pd.read_csv("MRTstations.csv")
sg_mrt.head()

,MRTstations
0,Yio Chu Kang
1,Ang Mo Kio
2,Tanah Merah
3,Bedok
4,Kembangan


In [4]:
sg_mrt.shape

(122, 1)

### 3. Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(MRTstations):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Singapore'.format(MRTstations))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(MRTstations) for MRTstations in sg_mrt["MRTstations"].tolist() ]

In [10]:
coords

[[1.3879700000000526, 103.86879000000005],
 [1.3716100000000324, 103.84546000000006],
 [1.323500000000024, 103.94174000000004],
 [1.3242500000000632, 103.95297000000005],
 [1.3222800000000348, 103.91135000000008],
 [1.337500000000034, 103.90917000000007],
 [1.3344500000000608, 103.91865000000007],
 [1.342610000000036, 103.92539000000005],
 [1.3507900000000745, 103.85110000000009],
 [1.3487189831982467, 103.83914441051587],
 [1.349520000000041, 103.75277000000006],
 [1.3641700000000583, 103.75722000000007],
 [1.3787700000000314, 103.76977000000005],
 [1.3719288111334345, 103.7671988344929],
 [1.3621554615653328, 103.76587008156457],
 [1.2825000000000273, 103.83806000000004],
 [1.2895300000000702, 103.83208000000008],
 [1.2873400000000288, 103.81883992945394],
 [1.2652000000000498, 103.82010000000008],
 [1.2720200000000546, 103.80226000000005],
 [1.2750900000000343, 103.81980000000004],
 [1.3148821830051929, 103.80511276797928],
 [1.3119400000000496, 103.79333000000008],
 [1.341430000000

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
sg_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
sg_mrt['Latitude'] = sg_coords['Latitude']
sg_mrt['Longitude'] = sg_coords['Longitude']

In [13]:
# check the MRTstations and the coordinates
print(sg_mrt.shape)
sg_mrt

(122, 3)


,MRTstations,Latitude,Longitude
0,Yio Chu Kang,1.387970,103.868790
1,Ang Mo Kio,1.371610,103.845460
2,Tanah Merah,1.323500,103.941740
3,Bedok,1.324250,103.952970
4,Kembangan,1.322280,103.911350
5,Kaki Bukit,1.337500,103.909170
6,Bedok North,1.334450,103.918650
7,Bedok Reservoir,1.342610,103.925390
8,Bishan,1.350790,103.851100
9,Marymount,1.348719,103.839144


In [14]:
# save the DataFrame as CSV file
sg_mrt.to_csv("sg_mrt.csv", index = False)

### 4. Create a map of Singapore with MRT stations superimposed on top

In [15]:
# get the coordinates of Singapore
address = 'Singapore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore is 1.357107, 103.8194992.


In [16]:
# create map of Singapore using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, mrtStation in zip(sg_mrt['Latitude'], sg_mrt['Longitude'], sg_mrt['MRTstations']):
    label = '{}'.format(mrtStation)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sg)  
    
map_sg

In [17]:
# save the map as HTML file
map_sg.save('map_sg.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [1]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET: your Foursquare Secret


In [26]:
radius = 250
LIMIT = 150

venues = []

for lat, long, mrtStation in zip(sg_mrt['Latitude'], sg_mrt['Longitude'], sg_mrt['MRTstations']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            mrtStation,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['MRT Station', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1763, 7)


,MRT Station,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Yio Chu Kang,1.38797,103.86879,Cedele,1.387515,103.869112,Diner
1,Yio Chu Kang,1.38797,103.86879,Awfully Chocolate,1.387588,103.869206,Dessert Shop
2,Yio Chu Kang,1.38797,103.86879,Cold Storage,1.387460,103.869480,Supermarket
3,Yio Chu Kang,1.38797,103.86879,Mai Thai,1.387782,103.869318,Thai Restaurant
4,Yio Chu Kang,1.38797,103.86879,Toast Box 土司工坊 (Toast Box),1.387929,103.869063,Café


In [28]:
venues_df.groupby(["MRT Station"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
MRT Station,,,,,,
Admiralty,5,5,5,5,5,5
Aljunied,5,5,5,5,5,5
Ang Mo Kio,17,17,17,17,17,17
Bartley,1,1,1,1,1,1
Bayfront,19,19,19,19,19,19
Beauty World,52,52,52,52,52,52
Bedok,1,1,1,1,1,1
Bedok North,4,4,4,4,4,4
Bedok Reservoir,1,1,1,1,1,1


In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 229 uniques categories.


In [30]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Diner', 'Dessert Shop', 'Supermarket', 'Thai Restaurant', 'Café',
       'Shopping Mall', 'Pharmacy', 'Japanese Restaurant', 'Gym',
       'Hostel', 'Cantonese Restaurant', 'Noodle House',
       'Chinese Restaurant', 'Food Court', 'Modern European Restaurant',
       'Halal Restaurant', 'Malay Restaurant', 'Fast Food Restaurant',
       'Electronics Store', 'Bank', 'Breakfast Spot', 'Pizza Place',
       'Vegetarian / Vegan Restaurant', 'Sandwich Place', 'Pet Store',
       'Steakhouse', 'Seafood Restaurant', 'Coffee Shop', 'Bus Station',
       'Train Station', 'Playground', 'Reservoir', 'Shaanxi Restaurant',
       'Bubble Tea Shop', 'Department Store', 'Asian Restaurant',
       'Outdoors & Recreation', 'Food Truck', 'Basketball Court',
       'Paper / Office Supplies Store', 'Karaoke Bar', 'Pool',
       'Bed & Breakfast', 'Italian Restaurant', 'Beer Garden',
       'Thrift / Vintage Store', 'Bistro', 'Ice Cream Shop', 'Bus Stop',
       'Soup Place'], dtype=object)

In [31]:
# check if the results contain "Bubble Tea Shop"
"mrtStation" in venues_df['VenueCategory'].unique()

False

### 6. Analyze each MRT station

In [32]:
# one hot encoding
sg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['MRT Station'] = venues_df['MRT Station'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

print(sg_onehot.shape)
sg_onehot.head()

(1763, 230)


,MRT Station,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cantonese Restaurant,Casino,Chinese Breakfast Place,Chinese Restaurant,Church,Churrascaria,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel Lounge,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Yio Chu Kang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Yio Chu Kang,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
sg_grouped = sg_onehot.groupby(["MRT Station"]).mean().reset_index()

print(sg_grouped.shape)
sg_grouped

(113, 230)


,MRT Station,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cantonese Restaurant,Casino,Chinese Breakfast Place,Chinese Restaurant,Church,Churrascaria,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel Lounge,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Admiralty,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.

In [34]:
len(sg_grouped[sg_grouped["Bubble Tea Shop"] > 0])

13

In [35]:
sg_bbt = sg_grouped[["MRT Station","Bubble Tea Shop"]]

In [36]:
sg_bbt.head()

,MRT Station,Bubble Tea Shop
0,Admiralty,0.0
1,Aljunied,0.0
2,Ang Mo Kio,0.0
3,Bartley,0.0
4,Bayfront,0.0


In [77]:
# set number of clusters
kclusters = 3

sg_clustering = sg_bbt.drop(["MRT Station"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [78]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sg_merged = sg_bbt.copy()

# add clustering labels
sg_merged["Cluster Labels"] = kmeans.labels_

In [79]:
sg_merged.rename(columns={"MRT Station": "MRT Station"}, inplace=True)
sg_merged.head()

,MRT Station,Bubble Tea Shop,Cluster Labels
0,Admiralty,0.0,0
1,Aljunied,0.0,0
2,Ang Mo Kio,0.0,0
3,Bartley,0.0,0
4,Bayfront,0.0,0


In [80]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sg_merged = sg_merged.join(sg_mrt.set_index("MRTstations"), on = "MRT Station")

print(sg_merged.shape)
sg_merged.head() # check the last columns!

(113, 5)


,MRT Station,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Admiralty,0.0,0,1.440690,103.800930
1,Aljunied,0.0,0,1.329278,103.879562
2,Ang Mo Kio,0.0,0,1.371610,103.845460
3,Bartley,0.0,0,1.343927,103.875457
4,Bayfront,0.0,0,1.281190,103.858730


In [81]:
# sort the results by Cluster Labels
print(sg_merged.shape)
sg_merged.sort_values(["Cluster Labels"], inplace=True)
sg_merged

(113, 5)


,MRT Station,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Admiralty,0.000000,0,1.440690,103.800930
82,Redhill,0.000000,0,1.287340,103.818840
81,Raffles Place,0.000000,0,1.281890,103.849120
80,Queenstown,0.000000,0,1.299660,103.801720
79,Punggol,0.000000,0,1.402460,103.906860
78,Promenade,0.000000,0,1.294010,103.860320
77,Potong Pasir,0.000000,0,1.333600,103.867170
76,Paya Lebar,0.000000,0,1.325030,103.890490
75,Pasir Ris,0.000000,0,1.371940,103.949940
74,Pasir Panjang,0.000000,0,1.292290,103.768190


In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['MRT Station'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

In [83]:
sg_merged.loc[sg_merged['Cluster Labels'] == 0]

,MRT Station,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Admiralty,0.000000,0,1.440690,103.800930
82,Redhill,0.000000,0,1.287340,103.818840
81,Raffles Place,0.000000,0,1.281890,103.849120
80,Queenstown,0.000000,0,1.299660,103.801720
79,Punggol,0.000000,0,1.402460,103.906860
78,Promenade,0.000000,0,1.294010,103.860320
77,Potong Pasir,0.000000,0,1.333600,103.867170
76,Paya Lebar,0.000000,0,1.325030,103.890490
75,Pasir Ris,0.000000,0,1.371940,103.949940
74,Pasir Panjang,0.000000,0,1.292290,103.768190


In [84]:
sg_merged.loc[sg_merged['Cluster Labels'] == 1]

,MRT Station,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
60,Lavender,0.040000,1,1.30717,103.86301
61,Little India,0.024390,1,1.31107,103.85483
49,Jurong East,0.024390,1,1.33437,103.74367
29,Clarke Quay,0.034483,1,1.29133,103.84584
40,Fort Canning,0.032258,1,1.29756,103.84716
35,Downtown,0.040000,1,1.37716,103.95553
33,Dhoby Ghaut,0.031250,1,1.29874,103.84800
25,Chinatown,0.018182,1,1.28479,103.84419
28,City Hall,0.030303,1,1.29257,103.85687


In [85]:
sg_merged.loc[sg_merged['Cluster Labels'] == 2]

,MRT Station,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
58,Labrador Park,0.055556,2,1.27202,103.80226
27,Choa Chu Kang,0.058824,2,1.38616,103.74618
11,Bishan,0.062500,2,1.35079,103.85110
